# Azure databricks notebook
Keep in mind that this notebook does not work as a standalone piece of code. Databricks automatically creates spark session, which is not created here.

In [ ]:
# import the needed libraries
import requests
import pandas as pd
import ast
import datetime
import os
import time

In [ ]:
# define a function for getting the data from API
def get_weather_info(city_name, lat, lon, api_key = "<API key>"):
    result = requests.get(f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={api_key}")
    print(f"{city_name}: {result}")
    result = result.__dict__['_content']
    result = result.decode("UTF-8")
    result = ast.literal_eval(result)
    return result

In [ ]:
# define a function for extracting the needed data from binary dictionary
def transform_dict(input, city_name):
    comps = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']
    
    result = {
        'datetime': datetime.datetime.fromtimestamp(input['list'][0]['dt']),
        'city_name': city_name,
        'components': input['list'][0]['components']
    }

    for comp in comps:
        result[comp] = result['components'][comp]
    
    return result

In [ ]:
# create list of cities with their coordinates
cities = [
    {'name': "Istanbul", 'coords': [41.013611, 28.955]},
    {'name': "Moscow", 'coords': [55.75, 37.616667]},
    {'name': "London", 'coords': [51.504170004107046, -0.1194611248711073]},
    {'name': "St Petersburg", 'coords': [59.93110937948638, 30.363516140439522]},
    {'name': "Berlin", 'coords': [52.51170651354087, 13.41361710195995]},
    {'name': "Madrid", 'coords': [40.4144891173105, -3.694686505477918]},
    {'name': "Rome", 'coords': [41.89729488284909, 12.49582913148365]},
    {'name': "Kiev", 'coords': [50.44917520587561, 30.529807429835913]},
    {'name': "Paris", 'coords': [48.856579934739976, 2.346998121599381]},
    {'name': "Bucharest", 'coords': [44.42264357429106, 26.097343584279603]},
    {'name': "Budapest", 'coords': [47.4993022596122, 19.04374762284502]},
    {'name': "Hamburg", 'coords': [53.54833073502459, 9.986840272009005]},
    {'name': "Minsk", 'coords': [53.899364736590755, 27.55756696023462]},
    {'name': "Warsaw", 'coords': [52.22827518265219, 21.00760182280719]},
    {'name': "Belgrade", 'coords': [44.80892774759441, 20.46375548518806]},
    {'name': "Vienna", 'coords': [48.20465870424358, 16.374529363925667]},
    {'name': "Kharkiv", 'coords': [49.98971197425672, 36.245963362740795]},
    {'name': "Barcelona", 'coords': [41.396968059851055, 2.177316056003728]},
    {'name': "Novosibirsk", 'coords': [54.98765966860813, 82.8903219850264]},
    {'name': "Nizhny Novogorod", 'coords': [56.31789365348282, 43.9419231781059]},
    {'name': "Milan", 'coords': [45.46309744125095, 9.188021882331142]},
    {'name': "Yekaterinburg", 'coords': [56.84020986982188, 60.603536657093244]},
    {'name': "Munich", 'coords': [48.134282078968155, 11.585743582754201]},
    {'name': "Prague", 'coords': [50.086015365953436, 14.440253312897223]},
    {'name': "Samara", 'coords': [53.20927739318431, 50.14987747329034]},
    {'name': "Omsk", 'coords': [54.96518430411747, 73.34778076649457]},
    {'name': "Sofia", 'coords': [42.689914669201336, 23.322531481692884]},
    {'name': "Dnipropetrovsk", 'coords': [48.46746625145549, 35.05292890591869]},
    {'name': "Kazan", 'coords': [55.785291497170974, 49.12851061005173]},
    {'name': "Ufa", 'coords': [54.7330200694228, 55.95725745542502]},
    {'name': "Chelyabinsk", 'coords': [55.16471277893009, 61.42881133336607]},
    {'name': "Donetsk", 'coords': [48.01130562373803, 37.80286486289703]},
    {'name': "Naples", 'coords': [40.85144733241689, 14.266546857532795]},
    {'name': "Birmingham", 'coords': [52.481841355258105, -1.8870111399839626]},
    {'name': "Perm", 'coords': [57.99841737852766, 56.23691605155155]},
    {'name': "Rostov-on-Don", 'coords': [47.23436436304084, 39.711283145842586]},
    {'name': "Odessa", 'coords': [46.479118559117225, 30.726787066311267]},
    {'name': "Volgograd", 'coords': [48.705308481348396, 44.513536915351665]},
    {'name': "Cologne", 'coords': [50.93734081251699, 6.9549276730997915]},
    {'name': "Turin", 'coords': [45.071915985442104, 7.684522816833888]},
    {'name': "Voronezh", 'coords': [51.666656734979604, 39.190653585642856]},
    {'name': "Krasnoyarsk", 'coords': [56.01744305703123, 92.88112871952929]},
    {'name': "Saratov", 'coords': [51.54331878590676, 46.0135608113927]},
    {'name': "Zagreb", 'coords': [45.810647035231554, 15.979847313846996]},
    {'name': "Zaporizhzhia", 'coords': [47.83708916309367, 35.14402046526135]},
    {'name': "Lodz", 'coords': [51.75903368477017, 19.457811119128234]},
    {'name': "Marseille", 'coords': [43.29393854629416, 5.3676434211237725]},
    {'name': "Riga", 'coords': [56.955992562056245, 24.103090910411698]},
    {'name': "Lviv", 'coords': [49.8335278676239, 24.02818652880955]},
    {'name': "Athens", 'coords': [37.9835026621704, 23.728208475318645]},
    {'name': "Thessaloniki", 'coords': [40.63839776688403, 22.942774390831342]},
    {'name': "Stockholm", 'coords': [59.32854920599373, 18.06680133759516]},
    {'name': "Kraków", 'coords': [50.060849718993204, 19.94509903911081]},
    {'name': "Valencia", 'coords': [39.4683822821822, -0.37570069974561665]},
    {'name': "Amsterdam", 'coords': [52.36658669527081, 4.896396253589842]},
    {'name': "Leeds", 'coords': [53.79906432403618, -1.5440081004778312]},
    {'name': "Tolyatti", 'coords': [53.51190716823151, 49.40654222437237]},
    {'name': "Kryvyi Rih", 'coords': [47.9087533762273, 33.38648188708254]},
    {'name': "Seville", 'coords': [37.386714915856395, -5.988029448910386]},
    {'name': "Palermo", 'coords': [38.11450213949013, 13.362513798379831]},
    {'name': "Ulyanovsk", 'coords': [54.3180665267727, 48.39067818929474]},
    {'name': "Kishinev", 'coords': [47.00958458756508, 28.86640571813421]},
    {'name': "Genoa", 'coords': [44.405698660586935, 8.948615502876631]},
    {'name': "Izhevsk", 'coords': [56.86187597001293, 53.222574052415425]},
    {'name': "Frankfurt", 'coords': [50.11194148347948, 8.67909661572005]},
    {'name': "Krasnodar", 'coords': [45.034002478560375, 38.978404612828015]},
    {'name': "Wroclaw", 'coords': [51.10764648727342, 17.036670313347358]},
    {'name': "Glasgow", 'coords': [55.86272888139115, -4.251718814287872]},
    {'name': "Yaroslavl", 'coords': [57.623661606535855, 39.887310705344035]},
    {'name': "Khabarovsk", 'coords': [48.48020525094305, 135.066724708675]},
    {'name': "Vladivostok", 'coords': [43.13315645233101, 131.91099362464485]},
    {'name': "Zaragoza", 'coords': [41.64832697047085, -0.8894906305173467]},
    {'name': "Essen", 'coords': [51.454189829312334, 7.01205901508707]},
    {'name': "Rotterdam", 'coords': [51.924587067371654, 4.477549549725197]},
    {'name': "Irkutsk", 'coords': [52.285044249346576, 104.28518224814201]},
    {'name': "Dortmund", 'coords': [51.51169812588056, 7.471597754944833]},
    {'name': "Stuttgart", 'coords': [48.77602613573551, 9.181294762393282]},
    {'name': "Barnaul", 'coords': [53.34379380555242, 83.78594132064147]},
    {'name': "Vilnius", 'coords': [54.68526366035061, 25.281305332525736]},
    {'name': "Poznan", 'coords': [52.40593708865194, 16.92772110407561]},
    {'name': "Dusseldorf", 'coords': [51.22617981905155, 6.771177112950843]},
    {'name': "Novokuznetsk", 'coords': [53.76052054554526, 87.1218055025053]},
    {'name': "Lisbon", 'coords': [38.7217886159164, -9.139689772587582]},
    {'name': "Helsinki", 'coords': [60.16943411012721, 24.939319966333052]},
    {'name': "Malaga", 'coords': [36.72031828488467, -4.421732816160877]},
    {'name': "Bremen", 'coords': [53.076837834066104, 8.79636183016191]},
    {'name': "Sheffield", 'coords': [53.37987062204897, -1.4688431716281212]},
    {'name': "Sarajevo", 'coords': [43.856001904615766, 18.41071014049277]},
    {'name': "Penza", 'coords': [53.2237945555763, 45.00118307686343]},
    {'name': "Ryazan", 'coords': [54.608614519138, 39.71358653445411]},
    {'name': "Orenburg", 'coords': [51.76814289162626, 55.096323694247076]},
    {'name': "Naberezhnye Chelny", 'coords': [55.73021095559491, 52.388183093879775]},
    {'name': "Duisburg", 'coords': [51.43413049037996, 6.770492803128714]},
    {'name': "Lipetsk", 'coords': [52.60924339342072, 39.59815658180228]},
    {'name': "Hannover", 'coords': [52.373341401226185, 9.733968340368945]},
    {'name': "Mykolaiv", 'coords': [46.97443122773534, 31.994173594455017]},
    {'name': "Tula", 'coords': [54.20438587528549, 37.6164514172862]},
    {'name': "Oslo", 'coords': [59.91616973099104, 10.750753676377329]},
    {'name': "Tyumen", 'coords': [57.14998576347368, 65.57704612185711]},
    {'name': "Copenhagen", 'coords': [55.67547415324423, 12.580056584668982]},
    {'name': "Kemerovo", 'coords': [55.33969063261753, 86.06195649178275]},
]

In [ ]:
# get the API data and transform the records
# the time.sleep function is used to omit the 60 calls per minute limit
results = []

for city in cities:
    try:
        result = get_weather_info(city['name'], city['coords'][0], city['coords'][1])
        result = transform_dict(result, city['name'])
        results.append(result)
        time.sleep(1.5)
    except:
        result = {'datetime': datetime.now(),
                  'city_name': city['name'],
                  'co': "",
                  'no': "",
                  'no2': "",
                  'o3': "",
                  'so2': "",
                  'pm2_5': "",
                  'pm10': "",
                  'nh3': ""}
        results.append(result)
        time.sleep(1.5)

Istanbul: <Response [200]>
Moscow: <Response [200]>
London: <Response [200]>
St Petersburg: <Response [200]>
Berlin: <Response [200]>
Madrid: <Response [200]>
Rome: <Response [200]>
Kiev: <Response [200]>
Paris: <Response [200]>
Bucharest: <Response [200]>
Budapest: <Response [200]>
Hamburg: <Response [200]>
Minsk: <Response [200]>
Warsaw: <Response [200]>
Belgrade: <Response [200]>
Vienna: <Response [200]>
Kharkiv: <Response [200]>
Barcelona: <Response [200]>
Novosibirsk: <Response [200]>
Nizhny Novogorod: <Response [200]>
Milan: <Response [200]>
Yekaterinburg: <Response [200]>
Munich: <Response [200]>
Prague: <Response [200]>
Samara: <Response [200]>
Omsk: <Response [200]>
Sofia: <Response [200]>
Dnipropetrovsk: <Response [200]>
Kazan: <Response [200]>
Ufa: <Response [200]>
Chelyabinsk: <Response [200]>
Donetsk: <Response [200]>
Naples: <Response [200]>
Birmingham: <Response [200]>
Perm: <Response [200]>
Rostov-on-Don: <Response [200]>
Odessa: <Response [200]>
Volgograd: <Response [2

In [ ]:
# creating spark dataframes from newly gathered data and from previous notebooks calls (they are in a file now)
df_new = spark.createDataFrame(pd.DataFrame(results).drop('components', axis=1))
df_old = spark.read.option("header", "true").option("inferschema", "true").csv("/mnt/airpollution/clean_data.csv")

In [ ]:
# combine two datasets into one table
df_old = df_old.union(df_new)

In [ ]:
# write the data to a single file, then move the file with data out of the logs directory
df_old\
 .coalesce(1)\
 .write\
 .mode("overwrite")\
 .option("header", "true")\
 .format("com.databricks.spark.csv")\
 .save("/mnt/airpollution/logs")

# Get the name of the CSV file that was just saved to Azure blob storage (it starts with 'part-')
files = dbutils.fs.ls("/mnt/airpollution/logs")
output_file = [x for x in files if x.name.startswith("part-")]

# Move the CSV file from a sub-folder (logs) to the data folder
# While simultaneously changing the file name
dbutils.fs.mv(output_file[0].path, "%s/clean_data.csv" % "/mnt/airpollution")

True